In [1]:
import draw

import numpy as np
from collections import defaultdict, Counter
from faker import Faker
from tabulate import tabulate
fake = Faker()

In [2]:
import importlib
importlib.reload(draw)

<module 'draw' from '/Users/cpj/Documents/code/lss/draw/draw.py'>

## Draw Simulation Methods

In [3]:
def init_players(num_players):
    """
    Create a collection of players in the form [player_id, player_name, player_ranking]
    Arguments:
        num_players: Number of players to add to the draw
    Returns:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
    """
    players = []
    id_to_name, id_to_rank = defaultdict(lambda: 'Bye'), defaultdict(lambda: 1600)
    for n in range(num_players):
        p_id, p_name, p_rank = np.random.randint(1000000, 9999999), fake.name(), np.random.randint(1600, 2000)
        players.append(['a' + str(p_id), p_name, p_rank])
        id_to_name['a' + str(p_id)] = p_name
        id_to_rank['a' + str(p_id)] = p_rank
    return players, id_to_name, id_to_rank

In [4]:
def init_pairings(players):
    """
    Randomly create results for the first round
    Arguments:
        players: (n x 3) numpy array containing [player_id, player_name, player_ranking]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    for n in range(0, len(players), 2):
        a, b = players[n][0], players[n+1][0]
        result = [a, b, np.random.choice([a, b, None], 1, p=(0.45, 0.45, 0.1))[0]]
        results.append(result)
    return results

In [5]:
def run_round(pairings, rnd, p_draw=0):
    """
    Randomly create results for a round
    Arguments:
        pairings: List of pairings for the round to be simulated
            (n x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    p_win = (1 - p_draw) / 2
    BYE = 0
    for pairing in pairings:
        a, b, pod = pairing[0][1], pairing[1][1], pairing[2]
        if a[:3] == 'BYE':
            result = [a,b,b]
        elif b[:3] == 'BYE':
            result = [a,b,a]
        else:
            result = [a, b, np.random.choice([a, b, None], 1, p=(p_win, p_win, p_draw))[0]]
        results.append(result + [rnd, pod])
        
    return results

## Draw Printing Methods

In [47]:
def print_standings(standings, id_to_name):
    """
    Print standings for the round in table form
    """
    table = []
    for rank, player in enumerate(standings):
        table.append([player[0], id_to_name[player[0]], player[1], rank+1, player[2], player[5], player[6], player[7], player[8], player[3], player[4]])
    print('\n', tabulate(table, headers=['ID', 'Name', 'Wins', 'Standing', 'CTB', 'PML', 'OML', 'OCTB', 'RPLD', 'Pod', 'Pod Wins']), '\n')
    

def print_pairings(pairs, id_to_name, scores):
    """
    Print pairings for the round in table form
    """
    table = []
    for table_num, pair in enumerate(pairs):
        a, b, pod = pair[0], pair[1], pair[2]
        score = scores[a[0], b[0]]
        row = [table_num+1, pod, id_to_name[a[1]], a[1], int(a[0])+1, id_to_name[b[1]], b[1], int(b[0])+1, score]
        table.append(row)
    headers = ['Table', 'Pod', 'Player 1 - Name', 'ID', 'Rank', 'Player 2 - Name', 'ID', 'Rank', 'GOF']
    print('\n', tabulate(table, headers=headers), '\n')

def print_byes(pairs_h):
    byes = []
    for pairs in pairs_h:
        for pair in pairs:
            a, b = pair[0][1], pair[1][1]
            if a[:3] == 'BYE': byes += [[b], ['BYE']]
            if b[:3] == 'BYE': byes += [[a], ['BYE']]

    byes = Counter(np.reshape(byes, -1))

    if len(byes) > 0:
        print('----- BYES -----')
        for k, v in byes.items():
            if k[:3] == 'BYE':
                print('Byes', ': ', v)
            else:
                print(k, ': ', v)

## Run a sample Tournament
Create a list of players then run through a multi-round tournament and print the results
Each round prints:
- Goodness of fit score: Score for the pairings allocated in the round. The maximum possible score is 4. If a number < 0 comes out, something has probably gone wrong
- Standings: A table of standings of the players (based only on # of wins, no accounting for draws, op wins etc)
- Pairings: A table of pairings for the following round (including player name and standing so you can check if players are being paired with others nearby)

To show all the standings and pairings at once, make sure to select the cell, click on the "Cell" menu, select -> "Current Outputs" and then select "Toggle Scrolling"

The entire history of "standings", "splits", "scores", "allocations", and "pairs" is recorded in the lists *standings_h, splits_h, scores_h, alloc_h, pairs_h*
    
Note: Early stage demo only. This does not support:
- Odd numbers of players and byes [Done]
- Allocating pods [Done]
- Iterative sampling methods to improve draw quality [Done]
- Balancing tail end pod sizes [Partly Done]
- Variable size pods [Done]
- etc

In [98]:
NUMBER_OF_PLAYERS = 64
ROUNDS  = ['swiss1', 'swiss1', 'swiss1', 'pod1', 'pod1', 'pod1',
           'pod2', 'pod2', 'pod2', 'swiss1', 'swiss1', 'swiss1']
NUMBER_OF_ROUNDS = len(ROUNDS)
PROBABILITY_OF_DRAW = 0.02
NUMBER_OF_DROPS = 1

In [129]:
NUMBER_OF_PLAYERS = 5
ROUNDS  = ['swiss1', 'swiss1', 'swiss1']
NUMBER_OF_ROUNDS = len(ROUNDS)
PROBABILITY_OF_DRAW = 0.02
NUMBER_OF_DROPS = 0

In [130]:
# Create players
players, id_to_name, id_to_rank = init_players(NUMBER_OF_PLAYERS)

In [131]:
importlib.reload(draw)
results = []
standings_h, splits_h, scores_h, alloc_h, pairs_h = [], [], [], [], []

for rnd in range(NUMBER_OF_ROUNDS):
    isLastRound = rnd==len(ROUNDS)-1
    standings, splits, scores, alloc, pairs = draw.create_draw(players, results, ROUNDS,
                                                               mode='test', minimiseTiebreakDist=False, seed=None)
    #standings, splits, scores, alloc, pairs = draw.create_draw(players, results, ROUNDS,
    #                                                           mode='test', minimiseTiebreakDist=True, seed=None)

    for collection, value in zip([standings_h, splits_h, scores_h, alloc_h, pairs_h],
                                 [standings,   splits,   scores,   alloc,   pairs]):
        collection.append(value)
    n_players = len(standings)
    brackets = [n for n in range(n_players) if (standings[n,[1]] != standings[n-1,[1]]).any()] + [n_players]
    n_odd_brackets = sum(np.array(brackets) % 2)
    max_score = draw.Points.valid - 2 * n_odd_brackets * \
                (draw.Points.valid + draw.Points.overall_win_distance) / n_players
    point_freq = Counter([score for score in scores[alloc==1].astype(int)])

    print('----- Round', rnd + 1, '-----', )
    print('Goodness of Fit:', (scores * alloc).sum() / len(scores))
    print('Maximum GOF:', max_score)
    print('Frequencies: ' + ', '.join([str(int(k)) + 'pts: ' + str(int(point_freq[k] / 2)) 
                                 for k in sorted(point_freq.keys())[::-1]]))
    if NUMBER_OF_PLAYERS <= 64:
        print_standings(standings, id_to_name, )
        print_pairings(pairs, id_to_name, scores)
    result = run_round(pairs, rnd, p_draw=PROBABILITY_OF_DRAW)
    results += result
    
    for _ in range(NUMBER_OF_DROPS):
        drop_id = np.random.randint(len(players))
        del(players[drop_id])

if NUMBER_OF_PLAYERS <= 64:
    print('----- Final Standings -----')
    print_standings(draw.get_standings(players, results, ROUNDS + [ROUNDS[-1]]), id_to_name, )
    
point_freq = Counter([y for x in [s[a==1].astype(int) for s, a in zip(scores_h, alloc_h)] for y in x])
print('Overall Pairing Score Frequency: ', ', '.join([str(int(k)) + 'pts: ' + str(int(point_freq[k] / 2))
        for k in sorted(point_freq.keys())[::-1]]))
print_byes(pairs_h)

----- Round 1 -----
Goodness of Fit: 13.333333333333334
Maximum GOF: 13.333333333333334
Frequencies: 16pts: 2, 8pts: 1

 ID        Name               Wins    Standing    CTB    PML    OML    OCTB    RPLD    Pod    Pod Wins
--------  ---------------  ------  ----------  -----  -----  -----  ------  ------  -----  ----------
a7368685  Eric Phillips         0           1      0      0      0       0       0      1           0
a7129153  Alicia Ballard        0           2      0      0      0       0       0      1           0
a5372261  Vincent Gilbert       0           3      0      0      0       0       0      1           0
a4148024  William Gomez         0           4      0      0      0       0       0      1           0
a4364941  Natalie White         0           5      0      0      0       0       0      1           0
BYE1      Bye                  -1           6      0      1      1       0       0      1          -1 


   Table    Pod  Player 1 - Name    ID          Rank  Player

#### Test Code Area

In [10]:
#print('\n', tabulate(results, headers=['Player1', 'Player2', 'Winner', 'Round', 'Pod']))
print('\n', tabulate(draw.get_standings(players, results, ROUNDS + [ROUNDS[-1]]), headers=['Player', 'WINS', 'CTB', 'POD', 'PODW', 'PML', 'OML', 'OCTB', 'RPLD']))


 Player      WINS       CTB    POD    PODW       PML       OML      OCTB    RPLD
--------  ------  --------  -----  ------  --------  --------  --------  ------
a1116728      13  0.864702      1       0  0.133333  0.333333  0.656848      15
a6283698      12  0.812567      1       0  0.2       0.368889  0.609052      15
a5331390      12  0.796541      1       0  0.133333  0.337778  0.652913      15
a1565848      12  0.795725      1       0  0.2       0.4       0.576795      15
a5475765      11  0.744589      1       0  0.266667  0.346667  0.650647      15
a2686073      11  0.731534      1       0  0.266667  0.44      0.544927      15
a1261390      11  0.728342      1       0  0.266667  0.431111  0.543393      15
a4446348      11  0.727556      1       0  0.266667  0.408889  0.58696       15
a6556544      11  0.72734       1       0  0.266667  0.466667  0.524644      15
a1981094      11  0.72729       1       0  0.266667  0.435556  0.551405      15
a8256992      10  0.680664      1     

In [11]:
plr_ = [['TEST15', 'Player, Test', '?'], ['TEST14', 'Player, Test', '?'], ['TEST13', 'Player, Test', '?'], ['TEST12', 'Player, Test', '?'], ['TEST11', 'Player, Test', '?'], ['TEST10', 'Player, Test', '?'], ['TEST9', 'Player, Test', '?'], ['TEST8', 'Player, Test', '?'], ['TEST7', 'Player, Test', '?'], ['TEST6', 'Player, Test', '?'], ['TEST5', 'Player, Test', '?'], ['TEST4', 'Player, Test', '?'], ['TEST3', 'Player, Test', '?'], ['TEST2', 'Player, Test', '?'], ['TEST1', 'Player, Test', '?']]
rslt_ = [['TEST8', 'TEST15', 'TEST15', 0, 1], ['TEST7', 'TEST10', 'TEST10', 0, 1], ['TEST6', 'TEST11', 'TEST11', 0, 1], ['TEST5', 'TEST12', 'TEST12', 0, 1], ['TEST4', 'TEST13', 'TEST4', 0, 1], ['TEST3', 'TEST14', 'TEST3', 0, 1], ['TEST1', 'TEST9', 'TEST1', 0, 1], ['TEST2', 'None', 'TEST2', 0, 1]]
rnds_ = ['swiss', 'swiss'] 

print('\n', tabulate(draw.get_standings(plr_, rslt_, rnds_), headers=['Player', 'WINS', 'CTB', 'POD', 'PODW', 'PML', 'OML', 'OCTB', 'RPLD']))
#draw.create_draw(plr_, rslt_, rnds_, mode='test', minimiseTiebreakDist=isLastRound)


 Player      WINS    CTB    POD    PODW    PML    OML    OCTB    RPLD
--------  ------  -----  -----  ------  -----  -----  ------  ------
TEST2          1      1      1       0      0      0       0       0
TEST11         1      1      1       0      0      1       0       1
TEST3          1      1      1       0      0      1       0       1
TEST4          1      1      1       0      0      1       0       1
TEST1          1      1      1       0      0      1       0       1
TEST15         1      1      1       0      0      1       0       1
TEST10         1      1      1       0      0      1       0       1
TEST12         1      1      1       0      0      1       0       1
TEST8          0      0      1       0      1      0       1       1
TEST7          0      0      1       0      1      0       1       1
TEST13         0      0      1       0      1      0       1       1
TEST5          0      0      1       0      1      0       1       1
TEST14         0      0      1  

In [12]:
from pandas import read_csv
pairings = read_csv('./Data/Pairings2.csv')
rslt_ = []
plr_ = []
rnds_ = ['swiss', 'swiss', 'swiss', 'swiss', 'swiss', 'swiss']
for n, row in pairings.iterrows():
    if row[0] <= 2:
        rslt_.append([str(row[3]), str(row[5]) if row[4] != 'None' else 'None', str(row[3]) if row[6]=='1WIN' else str(row[5]) if row[6]=='2WIN' else None, row[0]-1, 1])
    if row[0] == 1:
        plr_.append([str(row[3]), str(row[2]), 0])
        plr_.append([str(row[5]), str(row[4]), 0])

In [111]:
plr_  = [['TEST78', 'Player, Test', '?'], ['TEST182', 'Player, Test', '?'], ['TEST17', 'Player, Test', '?'], ['TEST237', 'Player, Test', '?'], ['TEST68', 'Player, Test', '?'], ['TEST133', 'Player, Test', '?'], ['TEST188', 'Player, Test', '?'], ['TEST88', 'Player, Test', '?'], ['TEST230', 'Player, Test', '?'], ['TEST50', 'Player, Test', '?'], ['TEST108', 'Player, Test', '?'], ['TEST231', 'Player, Test', '?'], ['TEST152', 'Player, Test', '?'], ['TEST93', 'Player, Test', '?'], ['TEST38', 'Player, Test', '?'], ['TEST51', 'Player, Test', '?'], ['TEST103', 'Player, Test', '?'], ['TEST248', 'Player, Test', '?'], ['TEST208', 'Player, Test', '?'], ['TEST241', 'Player, Test', '?'], ['TEST95', 'Player, Test', '?'], ['TEST243', 'Player, Test', '?'], ['TEST117', 'Player, Test', '?'], ['TEST218', 'Player, Test', '?'], ['TEST122', 'Player, Test', '?'], ['TEST130', 'Player, Test', '?'], ['TEST25', 'Player, Test', '?'], ['TEST138', 'Player, Test', '?'], ['TEST91', 'Player, Test', '?'], ['TEST58', 'Player, Test', '?'], ['TEST39', 'Player, Test', '?'], ['TEST33', 'Player, Test', '?'], ['TEST47', 'Player, Test', '?'], ['TEST70', 'Player, Test', '?'], ['TEST166', 'Player, Test', '?'], ['TEST196', 'Player, Test', '?'], ['TEST162', 'Player, Test', '?'], ['TEST209', 'Player, Test', '?'], ['TEST13', 'Player, Test', '?'], ['TEST202', 'Player, Test', '?'], ['TEST80', 'Player, Test', '?'], ['TEST250', 'Player, Test', '?'], ['TEST200', 'Player, Test', '?'], ['TEST136', 'Player, Test', '?'], ['TEST105', 'Player, Test', '?'], ['TEST12', 'Player, Test', '?'], ['TEST232', 'Player, Test', '?'], ['TEST111', 'Player, Test', '?'], ['TEST206', 'Player, Test', '?'], ['TEST249', 'Player, Test', '?'], ['TEST119', 'Player, Test', '?'], ['TEST81', 'Player, Test', '?'], ['TEST73', 'Player, Test', '?'], ['TEST11', 'Player, Test', '?'], ['TEST219', 'Player, Test', '?'], ['TEST30', 'Player, Test', '?'], ['TEST26', 'Player, Test', '?'], ['TEST174', 'Player, Test', '?'], ['TEST1', 'Player, Test', '?'], ['TEST84', 'Player, Test', '?'], ['TEST149', 'Player, Test', '?'], ['TEST143', 'Player, Test', '?'], ['TEST160', 'Player, Test', '?'], ['TEST207', 'Player, Test', '?'], ['TEST5', 'Player, Test', '?'], ['TEST223', 'Player, Test', '?'], ['TEST71', 'Player, Test', '?'], ['TEST150', 'Player, Test', '?'], ['TEST135', 'Player, Test', '?'], ['TEST180', 'Player, Test', '?'], ['TEST92', 'Player, Test', '?'], ['TEST43', 'Player, Test', '?'], ['TEST170', 'Player, Test', '?'], ['TEST144', 'Player, Test', '?'], ['TEST98', 'Player, Test', '?'], ['TEST64', 'Player, Test', '?'], ['TEST156', 'Player, Test', '?'], ['TEST86', 'Player, Test', '?'], ['TEST194', 'Player, Test', '?'], ['TEST36', 'Player, Test', '?'], ['TEST247', 'Player, Test', '?'], ['TEST240', 'Player, Test', '?'], ['TEST145', 'Player, Test', '?'], ['TEST192', 'Player, Test', '?'], ['TEST90', 'Player, Test', '?'], ['TEST204', 'Player, Test', '?'], ['TEST153', 'Player, Test', '?'], ['TEST127', 'Player, Test', '?'], ['TEST173', 'Player, Test', '?'], ['TEST220', 'Player, Test', '?'], ['TEST7', 'Player, Test', '?'], ['TEST113', 'Player, Test', '?'], ['TEST37', 'Player, Test', '?'], ['TEST141', 'Player, Test', '?'], ['TEST100', 'Player, Test', '?'], ['TEST236', 'Player, Test', '?'], ['TEST190', 'Player, Test', '?'], ['TEST67', 'Player, Test', '?'], ['TEST193', 'Player, Test', '?'], ['TEST235', 'Player, Test', '?'], ['TEST57', 'Player, Test', '?'], ['TEST123', 'Player, Test', '?'], ['TEST65', 'Player, Test', '?'], ['TEST6', 'Player, Test', '?'], ['TEST66', 'Player, Test', '?'], ['TEST46', 'Player, Test', '?'], ['TEST104', 'Player, Test', '?'], ['TEST212', 'Player, Test', '?'], ['TEST176', 'Player, Test', '?'], ['TEST172', 'Player, Test', '?'], ['TEST210', 'Player, Test', '?'], ['TEST132', 'Player, Test', '?'], ['TEST225', 'Player, Test', '?'], ['TEST198', 'Player, Test', '?'], ['TEST183', 'Player, Test', '?'], ['TEST233', 'Player, Test', '?'], ['TEST214', 'Player, Test', '?'], ['TEST242', 'Player, Test', '?'], ['TEST226', 'Player, Test', '?'], ['TEST201', 'Player, Test', '?'], ['TEST109', 'Player, Test', '?'], ['TEST157', 'Player, Test', '?'], ['TEST151', 'Player, Test', '?'], ['TEST2', 'Player, Test', '?'], ['TEST163', 'Player, Test', '?'], ['TEST125', 'Player, Test', '?'], ['TEST44', 'Player, Test', '?'], ['TEST31', 'Player, Test', '?'], ['TEST107', 'Player, Test', '?'], ['TEST229', 'Player, Test', '?'], ['TEST181', 'Player, Test', '?'], ['TEST18', 'Player, Test', '?'], ['TEST155', 'Player, Test', '?'], ['TEST124', 'Player, Test', '?'], ['TEST184', 'Player, Test', '?'], ['TEST14', 'Player, Test', '?'], ['TEST165', 'Player, Test', '?'], ['TEST16', 'Player, Test', '?'], ['TEST75', 'Player, Test', '?'], ['TEST9', 'Player, Test', '?'], ['TEST185', 'Player, Test', '?'], ['TEST3', 'Player, Test', '?'], ['TEST179', 'Player, Test', '?'], ['TEST171', 'Player, Test', '?'], ['TEST115', 'Player, Test', '?'], ['TEST79', 'Player, Test', '?'], ['TEST61', 'Player, Test', '?'], ['TEST224', 'Player, Test', '?'], ['TEST128', 'Player, Test', '?'], ['TEST94', 'Player, Test', '?'], ['TEST96', 'Player, Test', '?'], ['TEST161', 'Player, Test', '?'], ['TEST213', 'Player, Test', '?'], ['TEST60', 'Player, Test', '?'], ['TEST189', 'Player, Test', '?'], ['TEST120', 'Player, Test', '?'], ['TEST211', 'Player, Test', '?'], ['TEST167', 'Player, Test', '?'], ['TEST131', 'Player, Test', '?'], ['TEST139', 'Player, Test', '?'], ['TEST238', 'Player, Test', '?'], ['TEST195', 'Player, Test', '?'], ['TEST205', 'Player, Test', '?'], ['TEST29', 'Player, Test', '?'], ['TEST74', 'Player, Test', '?'], ['TEST34', 'Player, Test', '?'], ['TEST244', 'Player, Test', '?'], ['TEST23', 'Player, Test', '?'], ['TEST35', 'Player, Test', '?'], ['TEST15', 'Player, Test', '?'], ['TEST42', 'Player, Test', '?'], ['TEST62', 'Player, Test', '?'], ['TEST99', 'Player, Test', '?'], ['TEST40', 'Player, Test', '?'], ['TEST140', 'Player, Test', '?'], ['TEST82', 'Player, Test', '?'], ['TEST187', 'Player, Test', '?'], ['TEST246', 'Player, Test', '?'], ['TEST27', 'Player, Test', '?'], ['TEST59', 'Player, Test', '?'], ['TEST114', 'Player, Test', '?'], ['TEST137', 'Player, Test', '?'], ['TEST41', 'Player, Test', '?'], ['TEST21', 'Player, Test', '?'], ['TEST121', 'Player, Test', '?'], ['TEST20', 'Player, Test', '?'], ['TEST8', 'Player, Test', '?'], ['TEST142', 'Player, Test', '?'], ['TEST164', 'Player, Test', '?'], ['TEST24', 'Player, Test', '?'], ['TEST134', 'Player, Test', '?'], ['TEST215', 'Player, Test', '?'], ['TEST178', 'Player, Test', '?'], ['TEST245', 'Player, Test', '?'], ['TEST116', 'Player, Test', '?'], ['TEST77', 'Player, Test', '?'], ['TEST106', 'Player, Test', '?'], ['TEST228', 'Player, Test', '?'], ['TEST56', 'Player, Test', '?'], ['TEST222', 'Player, Test', '?'], ['TEST63', 'Player, Test', '?'], ['TEST52', 'Player, Test', '?'], ['TEST148', 'Player, Test', '?'], ['TEST118', 'Player, Test', '?'], ['TEST83', 'Player, Test', '?'], ['TEST48', 'Player, Test', '?'], ['TEST55', 'Player, Test', '?'], ['TEST110', 'Player, Test', '?'], ['TEST216', 'Player, Test', '?'], ['TEST69', 'Player, Test', '?'], ['TEST154', 'Player, Test', '?'], ['TEST101', 'Player, Test', '?'], ['TEST146', 'Player, Test', '?'], ['TEST72', 'Player, Test', '?'], ['TEST102', 'Player, Test', '?'], ['TEST19', 'Player, Test', '?'], ['TEST28', 'Player, Test', '?'], ['TEST217', 'Player, Test', '?'], ['TEST221', 'Player, Test', '?'], ['TEST85', 'Player, Test', '?'], ['TEST159', 'Player, Test', '?'], ['TEST89', 'Player, Test', '?'], ['TEST54', 'Player, Test', '?'], ['TEST197', 'Player, Test', '?'], ['TEST186', 'Player, Test', '?'], ['TEST87', 'Player, Test', '?'], ['TEST234', 'Player, Test', '?'], ['TEST168', 'Player, Test', '?'], ['TEST169', 'Player, Test', '?'], ['TEST191', 'Player, Test', '?'], ['TEST158', 'Player, Test', '?'], ['TEST177', 'Player, Test', '?'], ['TEST97', 'Player, Test', '?'], ['TEST203', 'Player, Test', '?'], ['TEST45', 'Player, Test', '?'], ['TEST126', 'Player, Test', '?'], ['TEST227', 'Player, Test', '?'], ['TEST199', 'Player, Test', '?'], ['TEST53', 'Player, Test', '?'], ['TEST76', 'Player, Test', '?'], ['TEST129', 'Player, Test', '?'], ['TEST112', 'Player, Test', '?'], ['TEST239', 'Player, Test', '?'], ['TEST147', 'Player, Test', '?'], ['TEST49', 'Player, Test', '?'], ['TEST32', 'Player, Test', '?'], ['TEST22', 'Player, Test', '?'], ['TEST10', 'Player, Test', '?'], ['TEST4', 'Player, Test', '?']]
rslt_ = [['TEST133', 'TEST188', 'TEST133', 0, 1], ['TEST88', 'TEST230', 'TEST88', 0, 1], ['TEST175', 'TEST50', 'TEST175', 0, 1], ['TEST108', 'TEST231', 'TEST108', 0, 1], ['TEST152', 'TEST93', 'TEST152', 0, 1], ['TEST38', 'TEST179', 'TEST38', 0, 1], ['TEST171', 'TEST115', 'TEST171', 0, 1], ['TEST79', 'TEST61', 'TEST79', 0, 1], ['TEST224', 'TEST128', 'TEST224', 0, 1], ['TEST94', 'TEST96', 'TEST94', 0, 1], ['TEST161', 'TEST213', 'TEST161', 0, 1], ['TEST60', 'TEST189', 'TEST189', 0, 1], ['TEST120', 'TEST211', 'TEST211', 0, 1], ['TEST167', 'TEST131', 'TEST131', 0, 1], ['TEST139', 'TEST238', 'TEST238', 0, 1], ['TEST195', 'TEST205', 'TEST205', 0, 1], ['TEST29', 'TEST74', 'TEST74', 0, 1], ['TEST34', 'TEST244', 'TEST244', 0, 1], ['TEST23', 'TEST35', 'TEST35', 0, 1], ['TEST15', 'TEST42', 'TEST42', 0, 1], ['TEST62', 'TEST99', 'TEST99', 0, 1], ['TEST187', 'TEST246', 'TEST246', 0, 1], ['TEST27', 'TEST59', 'TEST59', 0, 1], ['TEST114', 'TEST137', 'TEST137', 0, 1], ['TEST41', 'TEST21', 'TEST21', 0, 1], ['TEST121', 'TEST20', 'TEST20', 0, 1], ['TEST8', 'TEST142', 'TEST142', 0, 1], ['TEST164', 'TEST24', 'TEST24', 0, 1], ['TEST134', 'TEST215', 'TEST215', 0, 1], ['TEST178', 'TEST245', 'TEST245', 0, 1], ['TEST116', 'TEST77', 'TEST116', 0, 1], ['TEST106', 'TEST228', 'TEST106', 0, 1], ['TEST56', 'TEST222', 'TEST56', 0, 1], ['TEST63', 'TEST52', 'TEST63', 0, 1], ['TEST148', 'TEST118', 'TEST148', 0, 1], ['TEST83', 'TEST48', 'TEST83', 0, 1], ['TEST55', 'TEST110', 'TEST55', 0, 1], ['TEST216', 'TEST51', 'TEST216', 0, 1], ['TEST103', 'TEST248', 'TEST103', 0, 1], ['TEST208', 'TEST241', 'TEST208', 0, 1], ['TEST95', 'TEST243', 'TEST95', 0, 1], ['TEST117', 'TEST218', 'TEST117', 0, 1], ['TEST122', 'TEST130', 'TEST122', 0, 1], ['TEST25', 'TEST138', 'TEST25', 0, 1], ['TEST91', 'TEST58', 'TEST91', 0, 1], ['TEST39', 'TEST33', 'TEST39', 0, 1], ['TEST47', 'TEST70', 'TEST47', 0, 1], ['TEST237', 'TEST166', 'TEST237', 0, 1], ['TEST196', 'TEST162', 'TEST196', 0, 1], ['TEST209', 'TEST13', 'TEST209', 0, 1], ['TEST202', 'TEST80', 'TEST80', 0, 1], ['TEST250', 'TEST200', 'TEST200', 0, 1], ['TEST136', 'TEST105', 'TEST136', 0, 1], ['TEST12', 'TEST232', 'TEST12', 0, 1], ['TEST111', 'TEST206', 'TEST206', 0, 1], ['TEST249', 'TEST119', 'TEST119', 0, 1], ['TEST81', 'TEST73', 'TEST81', 0, 1], ['TEST11', 'TEST219', 'TEST11', 0, 1], ['TEST30', 'TEST26', 'TEST26', 0, 1], ['TEST174', 'TEST1', 'TEST1', 0, 1], ['TEST84', 'TEST149', 'TEST84', 0, 1], ['TEST143', 'TEST160', 'TEST143', 0, 1], ['TEST207', 'TEST5', 'TEST5', 0, 1], ['TEST68', 'TEST223', 'TEST68', 0, 1], ['TEST71', 'TEST150', 'TEST71', 0, 1], ['TEST135', 'TEST180', 'TEST135', 0, 1], ['TEST92', 'TEST43', 'TEST92', 0, 1], ['TEST170', 'TEST144', 'TEST170', 0, 1], ['TEST98', 'TEST64', 'TEST98', 0, 1], ['TEST194', 'TEST36', 'TEST36', 0, 1], ['TEST156', 'TEST86', 'TEST156', 0, 1], ['TEST247', 'TEST240', 'TEST247', 0, 1], ['TEST145', 'TEST192', 'TEST192', 0, 1], ['TEST90', 'TEST204', 'TEST204', 0, 1], ['TEST153', 'TEST127', 'TEST127', 0, 1], ['TEST173', 'TEST220', 'TEST220', 0, 1], ['TEST7', 'TEST113', 'TEST113', 0, 1], ['TEST37', 'TEST141', 'TEST141', 0, 1], ['TEST100', 'TEST236', 'TEST236', 0, 1], ['TEST190', 'TEST67', 'TEST190', 0, 1], ['TEST193', 'TEST235', 'TEST193', 0, 1], ['TEST57', 'TEST123', 'TEST57', 0, 1], ['TEST65', 'TEST6', 'TEST65', 0, 1], ['TEST66', 'TEST46', 'TEST66', 0, 1], ['TEST104', 'TEST212', 'TEST104', 0, 1], ['TEST176', 'TEST172', 'TEST176', 0, 1], ['TEST210', 'TEST132', 'TEST210', 0, 1], ['TEST225', 'TEST198', 'TEST225', 0, 1], ['TEST183', 'TEST233', 'TEST183', 0, 1], ['TEST214', 'TEST242', 'TEST242', 0, 1], ['TEST226', 'TEST201', 'TEST201', 0, 1], ['TEST109', 'TEST157', 'TEST109', 0, 1], ['TEST151', 'TEST2', 'TEST151', 0, 1], ['TEST163', 'TEST125', 'TEST163', 0, 1], ['TEST44', 'TEST31', 'TEST31', 0, 1], ['TEST107', 'TEST229', 'TEST107', 0, 1], ['TEST181', 'TEST18', 'TEST181', 0, 1], ['TEST184', 'TEST14', 'TEST14', 0, 1], ['TEST165', 'TEST16', 'TEST165', 0, 1], ['TEST75', 'TEST9', 'TEST9', 0, 1], ['TEST185', 'TEST3', 'TEST3', 0, 1], ['TEST69', 'TEST154', 'TEST69', 0, 1], ['TEST101', 'TEST146', 'TEST146', 0, 1], ['TEST72', 'TEST102', 'TEST102', 0, 1], ['TEST19', 'TEST28', 'TEST19', 0, 1], ['TEST217', 'TEST221', 'TEST221', 0, 1], ['TEST22', 'TEST85', 'TEST22', 0, 1], ['TEST159', 'TEST89', 'TEST89', 0, 1], ['TEST197', 'TEST186', 'TEST186', 0, 1], ['TEST87', 'TEST234', 'TEST87', 0, 1], ['TEST168', 'TEST169', 'TEST169', 0, 1], ['TEST191', 'TEST158', 'TEST158', 0, 1], ['TEST177', 'TEST97', 'TEST177', 0, 1], ['TEST203', 'TEST45', 'TEST45', 0, 1], ['TEST126', 'TEST4', 'TEST4', 0, 1], ['TEST227', 'TEST199', 'TEST199', 0, 1], ['TEST53', 'TEST76', 'TEST53', 0, 1], ['TEST239', 'TEST32', 'TEST239', 0, 1], ['TEST10', 'TEST129', 'TEST10', 0, 1], ['TEST49', 'TEST112', 'TEST49', 0, 1], ['TEST40', 'TEST140', 'TEST140', 0, 1], ['TEST147', 'TEST17', 'TEST17', 0, 1], ['TEST78', 'TEST82', 'TEST78', 0, 1], ['TEST155', 'TEST124', 'None', 0, 1], ['TEST182', 'TEST54', 'None', 0, 1], ['TEST13', 'TEST240', 'TEST13', 1, 1], ['TEST235', 'TEST188', 'TEST235', 1, 1], ['TEST149', 'TEST230', 'TEST149', 1, 1], ['TEST46', 'TEST18', 'TEST46', 1, 1], ['TEST16', 'TEST129', 'TEST16', 1, 1], ['TEST48', 'TEST212', 'TEST48', 1, 1], ['TEST248', 'TEST77', 'TEST248', 1, 1], ['TEST73', 'TEST231', 'TEST73', 1, 1], ['TEST222', 'TEST213', 'TEST222', 1, 1], ['TEST198', 'TEST93', 'TEST198', 1, 1], ['TEST58', 'TEST167', 'TEST58', 1, 1], ['TEST195', 'TEST243', 'TEST195', 1, 1], ['TEST29', 'TEST128', 'TEST29', 1, 1], ['TEST157', 'TEST139', 'TEST157', 1, 1], ['TEST34', 'TEST32', 'TEST34', 1, 1], ['TEST218', 'TEST228', 'TEST218', 1, 1], ['TEST187', 'TEST70', 'TEST187', 1, 1], ['TEST118', 'TEST62', 'TEST118', 1, 1], ['TEST27', 'TEST33', 'TEST27', 1, 1], ['TEST234', 'TEST60', 'TEST234', 1, 1], ['TEST134', 'TEST114', 'TEST134', 1, 1], ['TEST229', 'TEST110', 'TEST229', 1, 1], ['TEST160', 'TEST52', 'TEST160', 1, 1], ['TEST86', 'TEST15', 'TEST86', 1, 1], ['TEST250', 'TEST2', 'TEST250', 1, 1], ['TEST249', 'TEST178', 'TEST249', 1, 1], ['TEST174', 'TEST120', 'TEST174', 1, 1], ['TEST30', 'TEST111', 'TEST30', 1, 1], ['TEST144', 'TEST115', 'TEST144', 1, 1], ['TEST85', 'TEST219', 'TEST85', 1, 1], ['TEST233', 'TEST232', 'TEST233', 1, 1], ['TEST145', 'TEST64', 'TEST145', 1, 1], ['TEST6', 'TEST179', 'TEST6', 1, 1], ['TEST90', 'TEST112', 'TEST90', 1, 1], ['TEST173', 'TEST8', 'TEST173', 1, 1], ['TEST132', 'TEST207', 'TEST132', 1, 1], ['TEST7', 'TEST130', 'TEST7', 1, 1], ['TEST28', 'TEST96', 'TEST28', 1, 1], ['TEST100', 'TEST82', 'TEST100', 1, 1], ['TEST37', 'TEST105', 'TEST37', 1, 1], ['TEST214', 'TEST43', 'TEST214', 1, 1], ['TEST226', 'TEST172', 'TEST226', 1, 1], ['TEST44', 'TEST121', 'TEST44', 1, 1], ['TEST184', 'TEST125', 'TEST184', 1, 1], ['TEST97', 'TEST194', 'TEST97', 1, 1], ['TEST75', 'TEST67', 'TEST75', 1, 1], ['TEST185', 'TEST202', 'TEST185', 1, 1], ['TEST101', 'TEST61', 'TEST101', 1, 1], ['TEST217', 'TEST166', 'TEST217', 1, 1], ['TEST159', 'TEST138', 'TEST159', 1, 1], ['TEST76', 'TEST241', 'TEST76', 1, 1], ['TEST197', 'TEST50', 'TEST197', 1, 1], ['TEST168', 'TEST150', 'TEST168', 1, 1], ['TEST191', 'TEST123', 'TEST191', 1, 1], ['TEST203', 'TEST164', 'TEST203', 1, 1], ['TEST126', 'TEST41', 'TEST126', 1, 1], ['TEST227', 'TEST23', 'TEST227', 1, 1], ['TEST147', 'TEST223', 'TEST147', 1, 1], ['TEST40', 'TEST72', 'TEST40', 1, 1], ['TEST124', 'TEST162', 'TEST124', 1, 1], ['TEST54', 'TEST154', 'TEST54', 1, 1], ['TEST155', 'TEST51', 'TEST155', 1, 1], ['TEST182', 'TEST180', 'TEST182', 1, 1], ['TEST42', 'TEST153', 'TEST42', 1, 1], ['TEST88', 'TEST201', 'TEST88', 1, 1], ['TEST152', 'TEST221', 'TEST152', 1, 1], ['TEST244', 'TEST102', 'TEST244', 1, 1], ['TEST131', 'TEST38', 'TEST131', 1, 1], ['TEST113', 'TEST108', 'TEST113', 1, 1], ['TEST206', 'TEST169', 'TEST206', 1, 1], ['TEST215', 'TEST236', 'TEST215', 1, 1], ['TEST74', 'TEST79', 'TEST74', 1, 1], ['TEST99', 'TEST133', 'TEST99', 1, 1], ['TEST1', 'TEST242', 'TEST1', 1, 1], ['TEST200', 'TEST127', 'TEST200', 1, 1], ['TEST246', 'TEST161', 'TEST246', 1, 1], ['TEST20', 'TEST119', 'TEST20', 1, 1], ['TEST24', 'TEST80', 'TEST24', 1, 1], ['TEST63', 'TEST35', 'TEST63', 1, 1], ['TEST56', 'TEST21', 'TEST56', 1, 1], ['TEST83', 'TEST148', 'TEST83', 1, 1], ['TEST55', 'TEST31', 'TEST55', 1, 1], ['TEST216', 'TEST89', 'TEST216', 1, 1], ['TEST17', 'TEST211', 'TEST17', 1, 1], ['TEST95', 'TEST3', 'TEST95', 1, 1], ['TEST117', 'TEST106', 'TEST117', 1, 1], ['TEST122', 'TEST204', 'TEST122', 1, 1], ['TEST25', 'TEST36', 'TEST25', 1, 1], ['TEST237', 'TEST47', 'TEST237', 1, 1], ['TEST196', 'TEST238', 'TEST196', 1, 1], ['TEST136', 'TEST91', 'TEST136', 1, 1], ['TEST12', 'TEST171', 'TEST12', 1, 1], ['TEST26', 'TEST39', 'TEST26', 1, 1], ['TEST81', 'TEST192', 'TEST81', 1, 1], ['TEST84', 'TEST224', 'TEST84', 1, 1], ['TEST68', 'TEST11', 'TEST68', 1, 1], ['TEST141', 'TEST94', 'TEST141', 1, 1], ['TEST92', 'TEST9', 'TEST92', 1, 1], ['TEST98', 'TEST71', 'TEST98', 1, 1], ['TEST220', 'TEST245', 'TEST220', 1, 1], ['TEST247', 'TEST135', 'TEST247', 1, 1], ['TEST156', 'TEST5', 'TEST156', 1, 1], ['TEST45', 'TEST189', 'TEST45', 1, 1], ['TEST225', 'TEST193', 'TEST225', 1, 1], ['TEST210', 'TEST208', 'TEST210', 1, 1], ['TEST66', 'TEST59', 'TEST66', 1, 1], ['TEST176', 'TEST143', 'TEST176', 1, 1], ['TEST104', 'TEST137', 'TEST104', 1, 1], ['TEST57', 'TEST209', 'TEST57', 1, 1], ['TEST163', 'TEST158', 'TEST163', 1, 1], ['TEST151', 'TEST190', 'TEST151', 1, 1], ['TEST14', 'TEST183', 'TEST14', 1, 1], ['TEST107', 'TEST140', 'TEST107', 1, 1], ['TEST181', 'TEST205', 'TEST181', 1, 1], ['TEST165', 'TEST116', 'TEST165', 1, 1], ['TEST4', 'TEST65', 'TEST4', 1, 1], ['TEST186', 'TEST170', 'TEST186', 1, 1], ['TEST19', 'TEST109', 'TEST19', 1, 1], ['TEST177', 'TEST69', 'TEST177', 1, 1], ['TEST87', 'TEST22', 'TEST87', 1, 1], ['TEST53', 'TEST199', 'TEST53', 1, 1], ['TEST239', 'TEST146', 'TEST239', 1, 1], ['TEST10', 'TEST103', 'TEST10', 1, 1], ['TEST49', 'TEST142', 'TEST49', 1, 1], ['TEST78', 'TEST175', 'TEST78', 1, 1]]
rnds_ = ['swiss', 'swiss', 'swiss']

In [112]:
set(np.array(rslt_)[:,1]) - set(np.array(plr_)[:,0])

{'TEST175'}

In [113]:
importlib.reload(draw)
print('\n', tabulate(draw.get_standings(plr_, rslt_, rnds_), headers=['Player', 'WINS', 'CTB', 'POD', 'PODW', 'PML', 'OML', 'OCTB', 'RPLD']))

draw.create_draw(plr_, rslt_, rnds_)


 Player      WINS       CTB    POD    PODW    PML    OML      OCTB    RPLD
--------  ------  --------  -----  ------  -----  -----  --------  ------
TEST81         2  1             1       0    0     0.5   0.5            2
TEST117        2  1             1       0    0     0.5   0.5            2
TEST200        2  1             1       0    0     0.5   0.5            2
TEST177        2  1             1       0    0     0.5   0.5            2
TEST246        2  1             1       0    0     0.5   0.5            2
TEST113        2  1             1       0    0     0.5   0.5            2
TEST83         2  1             1       0    0     0.5   0.5            2
TEST215        2  1             1       0    0     0.5   0.5            2
TEST66         2  1             1       0    0     0.5   0.5            2
TEST74         2  1             1       0    0     0.5   0.5            2
TEST186        2  1             1       0    0     0.5   0.5            2
TEST165        2  1             1   

(array([['TEST81', 2, 1.0, ..., 0.5, 0.5, 2],
        ['TEST117', 2, 1.0, ..., 0.5, 0.5, 2],
        ['TEST200', 2, 1.0, ..., 0.5, 0.5, 2],
        ...,
        ['TEST219', 0, 0.0, ..., 0.5, 0.5, 2],
        ['TEST202', 0, 0.0, ..., 0.5, 0.5, 2],
        ['BYE1', -1, 0.0, ..., 1, 0, 0]], dtype=object),
 [0, 102, 142, 250],
 array([[-2048, 16, 16, ..., -16, -16, -56],
        [16, -2048, 16, ..., -16, -16, -56],
        [16, 16, -2048, ..., -16, -16, -56],
        ...,
        [-16, -16, -16, ..., -2048, 16, 8],
        [-16, -16, -16, ..., 16, -2048, 8],
        [-56, -56, -56, ..., 8, 8, -2048]], dtype=object),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 1, 0]]),
 [[(0, 'TEST81'), (61, 'TEST42'), 1],
  [(1, 'TEST117'), (46, 'TEST122'), 1],
  [(2, 'TEST200'), (34, 'TEST237'), 1],
  [(3, 'TEST177'), (26, 'TEST17'), 1],
  [(4, 'TE

In [14]:
a, b, c, d, e = draw.create_draw(plr_, rslt_, ['swiss', 'swiss', 'swiss'], minimiseTiebreakDist=True, seed=None)
c.astype(int), d

(array([[-2047,    17,  -503,     8,  -527,   -15],
        [   17, -2047,     8,     8,  -527,  -527],
        [ -503,     8, -2047,  -495,     8,     8],
        [    8,     8,  -495, -2047,     8,  -503],
        [ -527,  -527,     8,     8, -2047,    16],
        [  -15,  -527,     8,  -503,    16, -2047]]),
 array([[0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0]]))

In [15]:
(np.array([[0., 1., 0., 0., 0., 0.],
           [1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1.],
           [0., 0., 0., 0., 1., 0.],
           [0., 0., 0., 1., 0., 0.],
           [0., 0., 1., 0., 0., 0.]]) * c).sum()

66.19914829730988

In [16]:
(np.array([[0., 0., 0., 1., 0., 0.],
           [0., 0., 1., 0., 0., 0.],
           [0., 1., 0., 0., 0., 0.],
           [1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1.],
           [0., 0., 0., 0., 1., 0.]]) * c).sum()

65.50814907997847

Tournaments are separated into rounds. Every round, each player plays a match against another player, unless they have been awarded a bye. The result of each match (win, loss, or draw), is reported to the Tournament Organizer who enters it into GEM (Game Event Manager) software. During the tournament, players improve their standing by gaining match points as follows:

Win a match (W) = 1 match point
Lose a match (L) = 0 match points
Draw a match (D) = 0 match points (but is significant for tiebreakers)
Bye (B) = 1 match point (A bye is when a player is not assigned an opponent, usually because there is an uneven number of players in the tournament)
Information that is tracked throughout the tournament for each player includes:

Total match points = sum of all Wins and Byes
Cumulative tiebreaker (CTB) = tiebreaker last round / 4 + total match-win points
Player match loss % (PML) = Player total losses / total matches played
Player’s average opponent match loss % (OML%) = sum (PML of each opponent played so far) / number of opponents played
Player’s average opponent CTB (OCTB) = sum (CTB for each opponent played so far) / number of opponents played
Throughout the tournament, player standings (or rank) are determined by the following logic, in this order;

A player with higher total match-win points has a higher standing.
> If multiple players have the same number of match-win points, they tie for that standing, then;
A player with a higher CTB within a tie has the higher standing.
> If multiple players within a tie have the same CTB, then;
A player with a lower PML within a tie has the higher standing.
> If multiple players within a tie have the same PML, then;
A player with a lower OML% within a tie has the higher standing.
> If multiple players within a tie have the same OML%, then;
A player with a higher OCTB within a tie has the higher standing.
> If multiple players within a tie have the same OCTB, then;
Player standing is randomized within the tie.